In [6]:
import matplotlib.pyplot as plt
import pandas as pd
import folium

In [2]:
registros_filtrados = pd.read_excel("registros_filtrados.xlsx")
sensores = pd.read_excel("../sensores_airenuevoleon.xlsx")
pm25_validos = registros_filtrados[registros_filtrados["PM25"] >= 0].copy()
pm25 = pm25_validos["PM25"].dropna()

In [4]:
pm25_validos["anio"] = pd.to_datetime(pm25_validos["Dia"]).dt.year

# Filtrar eventos extremos
eventos_extremos = pm25_validos[pm25_validos["PM25"] >= 100].copy()

# Agrupar por sensor y año
resumen_mapa = eventos_extremos.groupby(["Sensor_id", "anio"]).agg({
    "PM25": ["count", "mean", "max"]
}).reset_index()

resumen_mapa.columns = ["Sensor_id", "Año", "Eventos", "PM25_promedio", "PM25_max"]

# Agregar ubicación
resumen_mapa = resumen_mapa.merge(sensores[["Sensor_id", "Zona", "Latitud", "Longitud"]],
                                   on="Sensor_id", how="left")



In [8]:
# Crear mapa base centrado en Monterrey
mapa = folium.Map(location=[25.67, -100.31], zoom_start=10)

# Crear subcapas por año
años_unicos = sorted(resumen_mapa["Año"].unique())

for año in años_unicos:
    grupo = folium.FeatureGroup(name=str(año))
    datos_año = resumen_mapa[resumen_mapa["Año"] == año]

    for _, fila in datos_año.iterrows():
        folium.CircleMarker(
            location=[fila["Latitud"], fila["Longitud"]],
            radius=5 + (fila["PM25_promedio"] / 20),
            popup=folium.Popup(f"""
                <b>Zona:</b> {fila['Zona']}<br>
                <b>Año:</b> {fila['Año']}<br>
                <b>Eventos:</b> {fila['Eventos']}<br>
                <b>PM2.5 promedio:</b> {fila['PM25_promedio']:.1f}<br>
                <b>PM2.5 máximo:</b> {fila['PM25_max']:.1f}
            """, max_width=250),
            color="red" if fila["PM25_promedio"] > 150 else "orange",
            fill=True,
            fill_opacity=0.7
        ).add_to(grupo)

    grupo.add_to(mapa)

# Agregar control de capas (años)
folium.LayerControl(collapsed=False).add_to(mapa)

# Mostrar mapa
mapa
